In [4]:
!pip install Faker

import csv
from faker import Faker
import random
from collections import defaultdict
import pandas as pd

In [5]:
df = pd.read_csv('/content/employees.csv')
print("Current columns:", list(df.columns))

Current columns: ['First Name', 'Last Name', 'Email', 'Phone', 'Gender', 'Age', 'Job Title', 'Years Of Experience', 'Salary', 'Department']


In [6]:
fake = Faker()
fake_in = Faker('hi_IN')  # India
fake_cn = Faker('zh_CN')  # Mainland China
fake_ca = Faker('en_CA')  # Canada
fake_kr = Faker('ko_KR')  # South Korea
fake_ph = Faker('fil_PH')  # Philippines
fake_tw = Faker('zh_TW')  # Taiwan
fake_mx = Faker('es_MX')  # Mexico

In [7]:
h1b_percentages = {
    'India': 0.729, 'China': 0.137, 'Canada': 0.024, 'South Korea': 0.019,
    'Philippines': 0.015, 'Taiwan': 0.009, 'Mexico': 0.008
}

languages = ['Spanish', 'Mandarin', 'French', 'Arabic', 'Russian', 'Japanese', 'German', 'Korean', 'Portuguese', 'Italian']

departments = {
    'Legal': 0.05, 'Marketing': 0.10, 'Administrative': 0.10, 'Operations': 0.20,
    'Sales': 0.10, 'Finance': 0.05, 'I/T': 0.10, 'Product': 0.20, 'Human Resource': 0.10
}

salary_ranges = {
    'Legal': (80000, 200000), 'Marketing': (40000, 110000), 'Administrative': (35000, 80000),
    'Operations': (50000, 150000), 'Sales': (40000, 120000), 'Finance': (60000, 180000),
    'I/T': (70000, 160000), 'Product': (80000, 200000), 'Human Resource': (50000, 120000)
}

In [11]:
def generate_name(gender, country):
    if country == 'USA':
        first_name = fake.first_name_male() if gender == 'M' else fake.first_name_female()
        last_name = fake.last_name()
    elif country == 'India': first_name, last_name = fake_in.first_name(), fake_in.last_name()
    elif country == 'China': first_name, last_name = fake_cn.first_name(), fake_cn.last_name()
    elif country == 'Canada': first_name, last_name = fake_ca.first_name(), fake_ca.last_name()
    elif country == 'South Korea': first_name, last_name = fake_kr.first_name(), fake_kr.last_name()
    elif country == 'Philippines': first_name, last_name = fake_ph.first_name(), fake_ph.last_name()
    elif country == 'Taiwan': first_name, last_name = fake_tw.first_name(), fake_tw.last_name()
    elif country == 'Mexico': first_name, last_name = fake_mx.first_name(), fake_mx.last_name()
    return first_name, last_name

In [12]:
def generate_employee(employee_id, gender):
    is_usa_citizen = random.random() > 0.4
    country = 'USA' if is_usa_citizen else random.choices(list(h1b_percentages.keys()), weights=list(h1b_percentages.values()))[0]
    first_name, last_name = generate_name(gender, country)

    ssn = fake.ssn() if is_usa_citizen else fake.unique.random_number(digits=9)
    department = random.choices(list(departments.keys()), weights=list(departments.values()))[0]
    salary_min, salary_max = salary_ranges[department]
    salary = round(random.uniform(salary_min, salary_max), 2)

    num_languages = random.choices([0, 1, 2], weights=[0.5, 0.3, 0.2])[0]
    employee_languages = random.sample(languages, num_languages)

    employee_data = {
        'First Name': first_name,
        'Last Name': last_name,
        'Email': fake.email(),
        'Phone': fake.phone_number(),
        'Gender': gender,
        'Age': random.randint(22, 65),
        'Job Title': fake.job(),
        'Years Of Experience': random.randint(0, 40),
        'Salary': salary,
        'Department': department,
        'Country': country,
        'SSN': ssn,
        'Languages': ','.join(employee_languages) if employee_languages else 'None'
    }

    return employee_data

In [13]:
employees = []
for i in range(1, 10001):
    gender = 'F' if i <= 5000 else 'M'
    employees.append(generate_employee(i, gender))

# Convert to DataFrame
synthetic_df = pd.DataFrame(employees)

# Reorder columns to match the original dataset, then add new columns
original_columns = df.columns.tolist()
new_columns = ['Country', 'SSN', 'Languages']
synthetic_df = synthetic_df[original_columns + new_columns]

print("Synthetic data columns:", list(synthetic_df.columns))
print("\nFirst few rows of synthetic data:")
print(synthetic_df.head(10))

# Save to CSV
synthetic_df.to_csv('synthetic_employees_10k.csv', index=False)
print("\n10,000 synthetic employee records generated and saved to 'synthetic_employees_10k.csv'.")

Synthetic data columns: ['First Name', 'Last Name', 'Email', 'Phone', 'Gender', 'Age', 'Job Title', 'Years Of Experience', 'Salary', 'Department', 'Country', 'SSN', 'Languages']

First few rows of synthetic data:
  First Name Last Name                       Email                   Phone  \
0     Shelly    Obrien       hthompson@example.net       857-683-8109x1242   
1      Donna  Gonzales         wrogers@example.com    001-586-443-3642x586   
2      Julie      Ford  elizabethgiles@example.org   001-447-589-4481x9306   
3      Linda    Stuart          usmith@example.com  001-243-454-1151x20985   
4     अद्वैत    नूरानी        joshua19@example.net            786.218.0437   
5      Donna      Hunt     isaiahsmith@example.com     (602)999-9687x98755   
6     Hannah  Mitchell     stevenbrown@example.net         +1-660-865-6135   
7        साई      दयाल     kathybarnes@example.net         +1-633-620-7136   
8       अशोक      मंडल         larry96@example.com        331-854-4773x022   
9      

In [14]:
df1 = pd.read_csv('/content/synthetic_employees_10k.csv')

In [15]:
gender_dept = df1.groupby('Department')['Gender'].value_counts().unstack()
print("\nGender distribution by department:")
print(gender_dept)


Gender distribution by department:
Gender            F    M
Department              
Administrative  486  509
Finance         239  247
Human Resource  516  558
I/T             514  480
Legal           242  245
Marketing       523  477
Operations      965  976
Product         997  980
Sales           518  528


In [16]:
current_employees = 320
scale_factor = 10000 / current_employees
employees_to_hire = gender_dept * (scale_factor - 1)

In [17]:
print("\nEmployees to hire by department and gender:")
print(employees_to_hire.astype(int))


Employees to hire by department and gender:
Gender              F      M
Department                  
Administrative  14701  15397
Finance          7229   7471
Human Resource  15609  16879
I/T             15548  14520
Legal            7320   7411
Marketing       15820  14429
Operations      29191  29524
Product         30159  29645
Sales           15669  15972


In [20]:
total_payroll = df1['Salary'].sum()
print(f"\nTotal yearly payroll for the new company: ${total_payroll:,.2f}")


Total yearly payroll for the new company: $1,022,123,179.52


In [21]:
avg_salary_dept = df1.groupby('Department')['Salary'].mean()
print("\nAverage salary by department:")
print(avg_salary_dept)


Average salary by department:
Department
Administrative     57311.634372
Finance           124058.371132
Human Resource     84178.605363
I/T               114869.753531
Legal             140869.746674
Marketing          74112.075520
Operations        101376.195945
Product           139781.701603
Sales              80671.238270
Name: Salary, dtype: float64
